
## ToDo:

*   Track sounds
*   







In [ ]:
import re
import os

In [ ]:
# Editable Variables
directory = "scene_files"

# Tracked variables
past_character_name = "[PAST_CHARACTER_NAME]"
menu_ref_character = None
current_file = None

# Output Lists
character_names_and_emotions = []
background_names = []
sound_names = []

In [ ]:
# Regex Sub Patterns
scenePattern = r'^## Scene: (.+)$'
generalText = r'.+?'
spaceCheck = r'(\s{3})?'
phoneCheck = r'\s*(\(Phone\))?\s*'
emotionCheck = r'(?:\((\w+)\))?'
b = r'\*\*'
q = r'["”“]'

# Text Placements
tab = "    "


In [ ]:
if not os.path.exists(directory):
  os.makedirs(directory)

In [ ]:
def replaceTextRules(input_line):
    # Removing all "\"
    input_line = input_line.replace("\\", "")

    # Bold formatting, getting list of all bolded strings
    bold_texts = re.findall(r'\*\*(.*?)\*\*', input_line)
    input_line = re.sub(r'\*\*(.*?)\*\*', r'{b}\1{/b}', input_line)

    # Italicizing
    input_line = re.sub(r'\*(.*?)\*', r'{i}\1{/i}', input_line)

    return input_line, bold_texts

In [ ]:
def lowercaseAndUnderscore(input_line):
    # Lowercase Line
    input_line = input_line.lower()

    # Underscore where there were spaces
    input_line = input_line.replace(" ", "_")

    return input_line

In [ ]:
def removeOddCharacters(input_line):
    # Removing all "\"
    input_line = input_line.replace("\\", "")

    # Removing all "#"
    input_line = input_line.replace("#", "")

    # Removing all Smart Quotes
    input_line = input_line.replace("’", "")
    input_line = input_line.replace("‘", "")
    input_line = input_line.replace("“", "")
    input_line = input_line.replace("”", "")

    return input_line

In [ ]:
#@title Menu Parser

def menuOptions(line):
    # Globals Used
    global current_file
    global past_character_name
    global menu_ref_character

    # Menu Options
    match = re.match(r'(\d+)\.\s*(.+)', line)
    if match:
        # Grab option number, and text, lowercase text and
        # remove * and whitespace
        option_number = match.group(1)
        option_text = match.group(2)
        option_text = option_text.replace("*", "")
        option_text = option_text.strip()

        # If option_number is 1, add menu syntax
        if option_number == "1":
            current_file.write(f"{tab}menu:\n")
            # Set character that was last used character
            menu_ref_character = past_character_name

        # Write option text
        current_file.write(f"{tab}{tab}\"{option_text}\":\n")

        # Set past_character to the menu ref character
        past_character_name = menu_ref_character

        # Line Parsed
        return True

    # Line Not Parsed
    return False


In [ ]:
#@title Background Parser

def backgroundLine(line):
    # Globals Used
    global past_character_name
    global current_file
    global background_names

    match = re.match(rf'{spaceCheck}\\\[Background: ({generalText})\\\]', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Grab background name, lowercase it,
        # and replace spaces with "_"
        background_name = match.group(2)
        background_name = lowercaseAndUnderscore(background_name)
        background_name = removeOddCharacters(background_name)

        # Adding background_name to names
        if (background_name not in background_names):
            background_names.append(background_name)

        # Print bg syntax
        current_file.write(f"{tab}{additional_spaces}scene bg {background_name}\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title Dialogue Parser

def characterLine(line):
    # Globals Used
    global past_character_name
    global current_file
    global character_names_and_emotions


    # Character dialogue
    match = re.match(rf'{spaceCheck}{phoneCheck}\*\*(.*?)\s*{emotionCheck}(?:\:{b}|{b}\:)\s*(.+)', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        phone = match.group(2)
        character_name = match.group(3)
        emotion = match.group(4)
        dialogue = match.group(5)

        # Text Cleaning
        character_name = lowercaseAndUnderscore(character_name)
        emotion = lowercaseAndUnderscore(emotion) if emotion else "[EMOTION]"
        dialogue, bold_text = replaceTextRules(dialogue)

        # Narator Line
        if character_name == "Narration" or character_name == "narration":
            # current_file.write(f"hide [{past_character_name}]\n")
            current_file.write(f"{tab}{additional_spaces}\"{dialogue}\"\n\n")
        # Character Line
        else:
            # If last character is not current character, need to hide and show
            if past_character_name != character_name:
                if (past_character_name != "[PAST_CHARACTER_NAME]"):
                    current_file.write(f"{tab}{additional_spaces}hide {past_character_name}\n")

                # Phone Line
                if (phone == "(Phone)" or phone == "(phone)"):
                    current_file.write(f"{tab}{additional_spaces}show {character_name} silhouette\n\n")
                    # Track Character/Silhouette Combo
                    character_emotion_combo = f"{character_name} silhouette"

                # Non-Phone Line
                else:
                    # Dialoge Line
                    current_file.write(f"{tab}{additional_spaces}show {character_name} {emotion}\n\n")
                    # Track Character/Emotion Combo
                    character_emotion_combo = f"{character_name} {emotion}"

                # Add Character/Emotion combo to output list if its not already
                if (character_emotion_combo not in character_names_and_emotions):
                    character_names_and_emotions.append(character_emotion_combo)

            current_file.write(f"{tab}{additional_spaces}{character_name} \"{dialogue}\"\n\n")

            # Set past_character
            past_character_name = character_name

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title Go To Parser

def gotoLine(line):
    # Globals Used
    global past_character_name
    global current_file

    # \[Go to Scene: Receive Plea Deal \#1\]

    # Goto Line
    match = re.match(rf'{spaceCheck}\\\[Go to {generalText}: ({generalText})\\\]', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Grab label name, lowercase it, remove \
        # and replace spaces with "_"
        label_name = match.group(2)
        label_name = lowercaseAndUnderscore(label_name)
        label_name = removeOddCharacters(label_name)

        # Print jump syntax
        current_file.write(f"{tab}{additional_spaces}jump {label_name}\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False


In [ ]:
#@title Notebook Entry Parser

def notebookLine(line):
    # Globals Used
    global past_character_name
    global current_file

    # \[Notebook Entry Added: “...”\]

    # Notebook entry Line
    # Adding closing quotes to the Regex breaks it, idk why, but this works anyways so...
    match = re.match(rf'{spaceCheck}\\\[Notebook Entry Added: {q}({generalText}){q}', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Grab Notebook entry
        notebook_entry = match.group(2)
        notebook_entry = replaceTextRules(notebook_entry)
        notebook_entry = notebook_entry[0]

        # Print notebook entry syntax
        current_file.write(f"{tab}{additional_spaces}$ notebook_entries.append(\"- {notebook_entry}\")\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title Glossary Entry Parser

def glossaryLine(line):
    # Globals Used
    global past_character_name
    global current_file

    # \[Glossary Entry Added: “...”\]

    # Glossary Entry Line
    # Adding closing quotes to the Regex breaks it, idk why, but this works anyways so...
    match = re.match(rf'{spaceCheck}\\\[Glossary Entry Added: {q}({generalText}){q}', line, re.IGNORECASE)
    if match:
        print("Match")
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Grab Glossary entry
        glossary_entry = match.group(2)
        glossary_entry = replaceTextRules(glossary_entry)
        glossary_entry = glossary_entry[0]

        # Print glossary entry syntax
        current_file.write(f"{tab}{additional_spaces}$ glossary_entries.append(\"{glossary_entry}\")\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title Show Map Parser

def showMapLine(line):
    # Globals Used
    global past_character_name
    global current_file

    # \[Show Map\]

    # Show Map Line
    match = re.match(rf'{spaceCheck}\\\[Show Map', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Print show map syntax
        current_file.write(f"{tab}{additional_spaces}call screen map()\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title Sound/Music/Voice Parser

def soundLine(line):
    # Globals Used
    global current_file
    global sound_names

    # \[Sound: Phone rings\]

    # Sound/Music/Voice Line
    match = re.match(rf'{spaceCheck}\\\[(play\s+)?(sound|music|voice):\s({generalText})\\\]', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Grab Sound Type, lowercase it
        sound_type = match.group(3);
        sound_type = sound_type.lower()

        # Grab the Sound Name, lowercase and underline it
        sound = match.group(4)
        sound = lowercaseAndUnderscore(sound)
        sound = removeOddCharacters(sound)

        # Track Sounds
        if sound not in sound_names:
            sound_names.append(sound)

        # Print Sound/Music/Voice syntax
        current_file.write(f"{tab}{additional_spaces}play {sound_type} {sound}\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title subLabel Parser

def subLableLine(line):
    # Globals Used
    global current_file

    # ### Subscene: Bus Talk

    # subLabel Line
    # Allows for any heading 2 label name, i.e : [HeadingName]: [LabelName]
    match = re.match(rf'{spaceCheck}###\s+{generalText}:\s(.+)', line, re.IGNORECASE)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

        # Grab label name, lowercase it, remove \
        # and replace spaces with "_"
        label_name = match.group(2)
        label_name = lowercaseAndUnderscore(label_name)
        label_name = removeOddCharacters(label_name)

        # Reset Past Character
        past_character_name = "[PAST_CHARACTER_NAME]"

        # Print Sound/Music/Voice syntax
        current_file.write(f"{additional_spaces}label {label_name}:\n\n")

        # Line Parsed
        return True

    # Line Not Parsed
    return False

In [ ]:
#@title Comment Line (Non-Parsed Line)

def commentLine(line):
    # Globals Used
    global current_file

    # Checking for Spaces at beginign
    match = re.match(rf'{spaceCheck}', line)
    if match:
        # Checking for spaces
        spaces = match.group(1) if match.group(1) else ""
        additional_spaces = " " * 8 if spaces else ""

    # Print show map syntax
    line = removeOddCharacters(line)
    current_file.write(f"{tab}{additional_spaces}# {line}\n\n")

In [ ]:
def transform_text(input_text, directory):
    # Split lines
    lines = [line for line in input_text.strip().split('\n') if line.strip()]

    global past_character_name
    global current_file

    past_character_name = "[PAST_CHARACTER_NAME]"
    current_file = None

    for line in lines:
        # Skip lines that are just ##
        if line.strip() == "##":
            continue

        ##################################################
        #
        # Scene / File Creation
        #
        ##################################################

        # Check for Scene or Ending
        if line.startswith("## Scene:") or line.startswith("## Ending:"):
            # If we have an open file, close it
            if current_file:
                current_file.close()

            # Determine whether it's a Scene or Ending
            if line.startswith("## Scene:"):
                label_name = line.replace('## Scene: ', '')
            elif line.startswith("## Ending:"):
                label_name = line.replace('## Ending: ', '')

            # Reset Past_Character
            past_character_name = "[PAST_CHARACTER_NAME]"

            label_name = lowercaseAndUnderscore(label_name)
            label_name = removeOddCharacters(label_name);

            # Create the filename
            filename = f"{directory}/{label_name}.rpy"
            current_file = open(filename, 'w')

            # Write the label and prefix (e.g., for scene or ending)
            current_file.write(f"label {label_name}:\n\n")

            continue

        ##################################################
        #
        # Line Processing
        #
        ##################################################

        # If we have an open file, process dialogues
        if current_file:
            # Menu Options
            if menuOptions(line):
                pass  # If True, skip the rest of the checks

            # Dialogue Line
            elif characterLine(line):
                pass  # If True, skip the rest of the checks

            # Background Line
            elif backgroundLine(line):
                pass  # If True, skip the rest of the checks

            # Goto Line
            elif gotoLine(line):
                pass  # If True, skip the rest of the checks

            # Notebook Entry Line
            elif notebookLine(line):
                pass  # If True, skip the rest of the checks

            # Glossary Entry Line
            elif glossaryLine(line):
                pass  # If True, skip the rest of the checks

            # Show Map Line
            elif showMapLine(line):
                pass  # If True, skip the rest of the checks

            # Sound/Music Line
            elif soundLine(line):
                pass # If True, skip the rest of the checks

            # SubLabel Line
            elif subLableLine(line):
                pass # If True, skip the rest of the checks

            # Comment Line (Line could not be parsed)
            else: commentLine(line)

    # Close the last file if it's open
    if current_file:
        current_file.close()

In [ ]:
def filename_file():
    # Global Variables
    global character_names_and_emotions
    global background_names
    global sound_names

    # Create filename file
    filename = f"filenames.txt"
    current_file = open(filename, 'w')

    # Add Background Names
    current_file.write(f"Background Names\n\n")
    for background in background_names:
        current_file.write(f"-{background}\n")
    current_file.write(f"\n")

    # Add Character/Emotion Combinations
    current_file.write(f"Character Emotion Combos\n\n")
    for character_emotion in character_names_and_emotions:
        current_file.write(f"-{character_emotion}\n")
    current_file.write(f"\n")

    # Add Sound Names
    current_file.write(f"Sounds\n\n")
    for sound in sound_names:
        current_file.write(f"-{sound}\n")

    current_file.close()

In [ ]:
# List all files in the directory
for filename in os.listdir('.'):
    # Check if the file has a .md extension
    if filename.endswith('.md'):
        file_path = os.path.join('.', filename)

        # Open and read the content of the .md file
        with open(file_path, 'r') as md_file:
            content = md_file.read()

        # Call the transform_text function with the content
        transform_text(content, directory)

# Create Filename File
filename_file()